Packages:
* [NLTK](http://www.nltk.org/howto/classify.html)
* [SpaCy](https://spacy.io/)
* [AllenNLP](https://allennlp.org/tutorials)

Articles:
* [A Comprehensive Guide to Understand and Implement Text Classification in Python](https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/)
* [Machine Learning, NLP: Text Classification using scikit-learn, python and NLTK](https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a)
* [State-of-the-Art Text Classification using BERT model: “Predict the Happiness” Challenge](https://appliedmachinelearning.blog/2019/03/04/state-of-the-art-text-classification-using-bert-model-predict-the-happiness-hackerearth-challenge/)

In [1]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from tqdm import tqdm
import numpy as np
import pandas as pd
import string
import time


stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    result_stemmed = []
    for token in simple_preprocess(text, min_len = 2):
        result.append(token)
#         if token not in STOPWORDS:
        result_stemmed.append(lemmatize_stemming(token))
    
    return (result, result_stemmed)

with open("./data/raw/Hygiene/hygiene.dat.labels") as f:
    LABELS = [int(l) for l in f.readlines() if l[0].isdigit()]

ALL_RAW_TEXTS = []
ALL_TEXTS = []
ALL_STEMMED_TEXTS = []
LABELED_TEXTS = []
LABELED_CONCAT_TEXTS = []
LABELED_STEMMED_TEXTS = []
LABELED_CONCAT_STEMMED_TEXTS = []

with open("./data/raw/Hygiene/hygiene.dat") as f:
    ALL_RAW_TEXTS = f.readlines()

for _text in tqdm(ALL_RAW_TEXTS):
    _result, _result_stemmed = preprocess(_text)
    ALL_TEXTS.append(_result)
    ALL_STEMMED_TEXTS.append(_result_stemmed)

LABELED_TEXTS = ALL_TEXTS[0:len(LABELS)]
LABELED_CONCAT_TEXTS = [" ".join(_text) for _text in ALL_TEXTS]

LABELED_STEMMED_TEXTS = ALL_STEMMED_TEXTS[0:len(LABELS)]
LABELED_CONCAT_STEMMED_TEXTS = [" ".join(_text) for _text in LABELED_STEMMED_TEXTS]

FEATURE_MORE =pd.read_csv("./data/raw/Hygiene/hygiene.dat.additional", header=None)

100%|██████████| 13299/13299 [06:21<00:00, 34.89it/s]


In [2]:
# create a dataframe using texts and lables
labeled_df = pd.DataFrame()
labeled_df['concat_stemmed_text'] = LABELED_CONCAT_STEMMED_TEXTS
labeled_df['stemmed_text'] = LABELED_STEMMED_TEXTS
labeled_df['label'] = LABELS

# split the dataset into training and validation datasets 
train_concat_stemmed_text, test_concat_stemmed_text, train_label, test_label = model_selection.train_test_split(labeled_df['concat_stemmed_text'], 
                                                                                  labeled_df['label'],
                                                                                  test_size = 0.2)
train_stemmed_text = labeled_df['stemmed_text'][train_concat_stemmed_text.index]
test_stemmed_text = labeled_df['stemmed_text'][test_concat_stemmed_text.index]

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_label = encoder.fit_transform(train_label)
test_label = encoder.fit_transform(test_label)

In [ ]:
# dictionary = corpora.Dictionary(processed_docs)
# print("Before prunn:%d"%(len(dictionary)))
# dictionary.filter_extremes(no_below = 2, no_above = 0.5)
# print("After prunn:%d"%(len(dictionary)))
# corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Count Vectors as features

In [4]:
class MyCountVectorizer:
    def __init__(self, max_df = 0.5):
        self.count_vect = CountVectorizer(analyzer='word', max_df = max_df)
        
    def fit(self, concat_stemmed_text):
        self.count_vect.fit(concat_stemmed_text)
    
    def transform(self, concat_stemmed_text):
        return self.count_vect.transform(concat_stemmed_text)

count_vect = MyCountVectorizer()
count_vect.fit(train_concat_stemmed_text)
train_count = count_vect.transform(train_concat_stemmed_text)
test_count =  count_vect.transform(test_concat_stemmed_text)

# # create a count vectorizer object 
# count_vect = CountVectorizer(analyzer='word', max_df = 0.5)
# count_vect.fit(train_concat_stemmed_text)

# # transform the training and validation data using count vectorizer object
# train_count =  count_vect.transform(train_concat_stemmed_text)
# test_count =  count_vect.transform(test_concat_stemmed_text)

# TF-IDF Vectors as features

In [13]:
%%time

# word level tf-idf
class MyTfidfVectorizer:
    def __init__(self, analyzer='word', ngram_range = None, max_features=5000):
        if ngram_range is None:
            self.tfidf_vect = TfidfVectorizer(analyzer = analyzer, max_features = max_features)
        else:
            self.tfidf_vect = TfidfVectorizer(analyzer = analyzer, ngram_range = ngram_range, 
                                              max_features = max_features)
            
    def fit(self, concat_stemmed_text):   
        self.tfidf_vect.fit(concat_stemmed_text)
        self.vocabulary = self.tfidf_vect.vocabulary_
        
    def transform(self, concat_stemmed_text):
        return self.tfidf_vect.transform(concat_stemmed_text)


tfidf_vect = MyTfidfVectorizer()
tfidf_vect.fit(train_concat_stemmed_text)
train_tfidf =  tfidf_vect.transform(train_concat_stemmed_text)
test_tfidf =  tfidf_vect.transform(test_concat_stemmed_text)

# ngram level tf-idf 
tfidf_vect_ngram = MyTfidfVectorizer(ngram_range=(2,3))
tfidf_vect_ngram.fit(train_concat_stemmed_text)
train_tfidf_ngram =  tfidf_vect_ngram.transform(train_concat_stemmed_text)
test_tfidf_ngram =  tfidf_vect_ngram.transform(test_concat_stemmed_text)

# characters level tf-idf
tfidf_vect_ngram_chars = MyTfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_vect_ngram_chars.fit(train_concat_stemmed_text)
train_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_concat_stemmed_text) 
test_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_concat_stemmed_text) 

CPU times: user 14.6 s, sys: 221 ms, total: 14.8 s
Wall time: 14.2 s


# Word Embeddings

## Build from review corpus

In [29]:
%%time

from gensim.models.fasttext import FastText

class MyFastTextTfidfVectorizer:
    def __init__(self, tfidf_vectorizer, EMBEDDING_SIZE = 100):
        self.embedding_size = EMBEDDING_SIZE
        self.tfidf_vectorizer = tfidf_vectorizer
        self.fasttext_model = FastText(size = EMBEDDING_SIZE, window = 5, min_count = 5)

    def tfidf2embedding(self, value_vector):
        _weighted_value = np.zeros(self.embedding_size)
        for key in self.tfidf_vectorizer.vocabulary:
            _index = self.tfidf_vectorizer.vocabulary[key]
            if value_vector[_index] != 0:
                _weighted_value += self.fasttext_model[key] * value_vector[_index]

        return _weighted_value
    
    def fit(self, stemmed_text):
        self.fasttext_model.build_vocab(sentences = stemmed_text)
        self.fasttext_model.train(sentences = stemmed_text, 
                                  total_examples = len(stemmed_text), 
                                  epochs=10)
        
    def transform(self, tfidf_values):
        return np.asarray([self.tfidf2embedding(_value.toarray()[0]) for _value in tfidf_values])

fasttext_tfidf_vect = MyFastTextTfidfVectorizer(tfidf_vect)
fasttext_tfidf_vect.fit(ALL_STEMMED_TEXTS)
train_fasttext_embedding = fasttext_tfidf_vect.transform(train_tfidf)
test_fasttext_embedding = fasttext_tfidf_vect.transform(test_tfidf)

AttributeError: 'MyTfidfVectorizer' object has no attribute 'vocabulary_'

## Prebuilt Embedding

In [ ]:
# %%time
# from keras.preprocessing import text, sequence
# from keras import layers, models, optimizers

# # load the pre-trained word-embedding vectors 
# embeddings_index = {}
# for i, line in enumerate(open('data/model/wiki-news-300d-1M.vec')):
#     values = line.split()
#     embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# # create a tokenizer 
# token = text.Tokenizer()
# token.fit_on_texts(LABELED_CONCAT_STEMMED_TEXTS)
# word_index = token.word_index

# # convert text to sequence of tokens and pad them to ensure equal length vectors 
# text_train_seq = sequence.pad_sequences(token.texts_to_sequences(train_text), maxlen=70)
# text_test_seq = sequence.pad_sequences(token.texts_to_sequences(test_text), maxlen=70)

# # create token-embedding mapping
# embedding_matrix = np.zeros((len(word_index) + 1, 300))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

# Text / NLP based features

In [ ]:
# %%time
# trainDF['char_count'] = trainDF['text'].apply(len)
# trainDF['word_count'] = trainDF['text'].apply(lambda x: len(x.split()))
# trainDF['word_density'] = trainDF['char_count'] / (trainDF['word_count']+1)
# trainDF['punctuation_count'] = trainDF['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation))) 
# trainDF['title_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
# trainDF['upper_case_word_count'] = trainDF['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
# %%time

# import textblob

# pos_family = {
#     'noun' : ['NN','NNS','NNP','NNPS'],
#     'pron' : ['PRP','PRP$','WP','WP$'],
#     'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
#     'adj' :  ['JJ','JJR','JJS'],
#     'adv' : ['RB','RBR','RBS','WRB']
# }

# # function to check and get the part of speech tag count of a words in a given sentence
# def check_pos_tag(x, flag):
#     cnt = 0
#     try:
#         wiki = textblob.TextBlob(x)
#         for tup in wiki.tags:
#             ppo = list(tup)[1]
#             if ppo in pos_family[flag]:
#                 cnt += 1
#     except:
#         pass
#     return cnt

# trainDF['noun_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'noun'))
# trainDF['verb_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'verb'))
# trainDF['adj_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adj'))
# trainDF['adv_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'adv'))
# trainDF['pron_count'] = trainDF['text'].apply(lambda x: check_pos_tag(x, 'pron'))

# Topic Models as features

In [ ]:
# # train a LDA Model
# lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
# X_topics = lda_model.fit_transform(text_train_count)
# topic_word = lda_model.components_
# vocab = count_vect.get_feature_names()

# # view the topic models
# n_top_words = 10
# topic_summaries = []
# for i, topic_dist in enumerate(topic_word):
#     topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
#     topic_summaries.append(' '.join(topic_words))

In [39]:
%%time

from gensim import corpora, models

class MyLDAVectorizer:
    def __init__(self, TOPIC_COUNT = 100):
        self.topic_count = TOPIC_COUNT
    
    def fit(self, stemmed_text):
        self.dictionary = corpora.Dictionary(stemmed_text)
        _corpus = [self.dictionary.doc2bow(_doc) for _doc in stemmed_text]
        self.tfidf_model = models.TfidfModel(_corpus)
        _tfidf_corpus = self.tfidf_model[_corpus]

        self.lda_model = models.LdaModel(_tfidf_corpus, 
                            num_topics = self.topic_count, 
                            id2word = self.dictionary,
                            random_state = 100,
                            eval_every = 5, 
                            alpha = 'auto', 
                            gamma_threshold = 0.01)
    
    def toarray(self, doc_topics):
        _doc_vect  = np.zeros((len(doc_topics), self.topic_count))
        
        for i, _doc in enumerate(doc_topics):
            for _topic, _weight in _doc:
                _doc_vect[i][_topic] = _weight
        
        return _doc_vect
        
    def transform(self, stemmed_text):
        _corpus = [self.dictionary.doc2bow(_doc) for _doc in stemmed_text]
        _tfidf_corpus = self.tfidf_model[_corpus]
        
        return self.toarray(self.lda_model[_tfidf_corpus])

lda_vect = MyLDAVectorizer()
lda_vect.fit(train_stemmed_text)
train_lda = lda_vect.transform(train_stemmed_text)
test_lda = lda_vect.transform(test_stemmed_text)

[(53, 0.029637955), (69, 0.8215249), (84, 0.026167497)]
[(1, 0.018453622), (19, 0.014443066), (24, 0.06439602), (27, 0.014569066), (50, 0.031918153), (53, 0.08823906), (54, 0.018961111), (56, 0.021068258), (57, 0.023602758), (58, 0.0106775), (60, 0.08043001), (65, 0.019430663), (67, 0.019447831), (68, 0.065047), (78, 0.039829627), (79, 0.019450113), (84, 0.030305486), (85, 0.013831024), (86, 0.107205816), (90, 0.20166728), (94, 0.012561857), (95, 0.029471038)]
[(24, 0.06931041), (25, 0.02184455), (27, 0.04055238), (38, 0.016704181), (44, 0.17726442), (50, 0.021686157), (53, 0.07355197), (54, 0.015965134), (55, 0.015223929), (57, 0.051597696), (60, 0.07752838), (65, 0.027182361), (67, 0.024037119), (68, 0.06915964), (78, 0.04404415), (84, 0.015891647), (85, 0.0162867), (86, 0.079128146)]
[(24, 0.08240327), (54, 0.57663494), (57, 0.013313533), (62, 0.02985379), (68, 0.061435036), (76, 0.02131703), (86, 0.10614495)]
[(9, 0.09476282), (24, 0.044865612), (27, 0.011094219), (29, 0.012208567)

[(19, 0.013294253), (24, 0.015408368), (50, 0.02821247), (53, 0.078807935), (56, 0.011743783), (57, 0.028242307), (60, 0.072803885), (62, 0.08607754), (65, 0.021229157), (67, 0.03434742), (68, 0.044708684), (75, 0.014833093), (78, 0.09412263), (84, 0.010394942), (85, 0.018331112), (86, 0.0994182), (92, 0.23990223)]
[(17, 0.8217745)]
[(8, 0.016422722), (24, 0.0223916), (38, 0.01629723), (50, 0.050900538), (53, 0.06955057), (57, 0.020511337), (60, 0.012493578), (74, 0.022660976), (78, 0.06759119), (83, 0.0397948), (86, 0.11504592), (96, 0.4218388)]
[(24, 0.05555176), (30, 0.04261648), (50, 0.01830494), (53, 0.09623122), (55, 0.011919171), (56, 0.024196113), (57, 0.08309788), (65, 0.029478488), (67, 0.03253681), (68, 0.04623644), (76, 0.013317207), (78, 0.053037886), (79, 0.2707939), (86, 0.12209905)]
[(15, 0.023833062), (24, 0.06251093), (37, 0.18479647), (55, 0.01191257), (57, 0.015071617), (58, 0.0108409505), (60, 0.12507284), (62, 0.031136252), (68, 0.13049436), (78, 0.048638884), (84

[(24, 0.027482038), (50, 0.49678022), (53, 0.067458846), (56, 0.013830607), (57, 0.015036499), (60, 0.03560322), (62, 0.025731266), (65, 0.028182197), (68, 0.06620066), (78, 0.0582531), (84, 0.025767364), (86, 0.065216444)]
[(19, 0.013257435), (41, 0.017522275), (50, 0.03999407), (51, 0.016139075), (53, 0.08585699), (54, 0.025845096), (56, 0.013172972), (58, 0.21658665), (60, 0.060703013), (65, 0.016427616), (67, 0.02881049), (68, 0.106580175), (70, 0.01479571), (76, 0.01231061), (78, 0.08039608), (84, 0.040456813), (86, 0.11102463), (95, 0.02370313)]
[(24, 0.061242983), (33, 0.049459606), (53, 0.1061767), (59, 0.38259447), (60, 0.051086195), (86, 0.23954621), (93, 0.015894953)]
[(86, 0.8729131)]
[(13, 0.38743353), (24, 0.025017565), (50, 0.011680275), (53, 0.0997512), (55, 0.011813202), (57, 0.02297273), (60, 0.07354752), (62, 0.013352922), (65, 0.028344166), (68, 0.04365518), (78, 0.037323017), (83, 0.015019721), (84, 0.055238962), (85, 0.015229217), (86, 0.083991826)]
[(24, 0.866508

[(5, 0.020011771), (16, 0.01279499), (24, 0.06912112), (41, 0.02580414), (50, 0.10122006), (53, 0.08610026), (55, 0.03569271), (57, 0.09183918), (58, 0.027152559), (60, 0.03306187), (68, 0.013234049), (70, 0.01291767), (78, 0.036547046), (84, 0.020168444), (85, 0.027245311), (86, 0.13472128), (95, 0.014833916), (97, 0.16139916)]
[(15, 0.07490687), (24, 0.80748594), (68, 0.028334029)]
[(19, 0.2505901), (24, 0.08653278), (41, 0.013028762), (50, 0.16322738), (53, 0.071048796), (54, 0.01976396), (55, 0.017633902), (57, 0.0120795155), (60, 0.06608645), (65, 0.06888396), (68, 0.023559257), (75, 0.010774464), (78, 0.039162282), (86, 0.071163274), (97, 0.010014983)]
[(24, 0.025591297), (31, 0.014129404), (50, 0.04136373), (53, 0.091715276), (57, 0.43446213), (60, 0.020458538), (76, 0.058570735), (78, 0.023731016), (83, 0.043101236), (84, 0.07709148), (86, 0.08036547)]
[(9, 0.15601325), (19, 0.023729717), (24, 0.054855037), (27, 0.019896064), (50, 0.018385403), (53, 0.059429895), (55, 0.0148920

[(19, 0.018196497), (24, 0.030103877), (50, 0.08402007), (53, 0.05747031), (57, 0.062207073), (60, 0.054241773), (68, 0.04127599), (70, 0.013779465), (74, 0.044396307), (78, 0.06417329), (85, 0.30499393), (86, 0.09916812), (97, 0.018764514)]
[(1, 0.010573765), (15, 0.18817364), (19, 0.038871024), (24, 0.0628405), (27, 0.011495949), (29, 0.011540146), (50, 0.030208055), (53, 0.07928771), (55, 0.019306408), (56, 0.025514085), (57, 0.05276695), (60, 0.071119875), (65, 0.012584799), (67, 0.025955176), (68, 0.05341025), (78, 0.0344892), (83, 0.01387076), (84, 0.03628262), (86, 0.10892822), (95, 0.016153354)]
[(8, 0.011155833), (24, 0.023037808), (27, 0.014809205), (50, 0.0355236), (53, 0.097448766), (54, 0.023009045), (55, 0.3609504), (57, 0.04329375), (60, 0.06381259), (67, 0.025335373), (68, 0.04127053), (76, 0.018400716), (78, 0.031294465), (84, 0.039452553), (85, 0.015863938), (86, 0.060136892), (95, 0.019125808)]
[(15, 0.037189174), (18, 0.013447515), (30, 0.30243328), (53, 0.1970191),

[(8, 0.012592483), (24, 0.05897436), (41, 0.011738606), (45, 0.024564082), (50, 0.061493684), (51, 0.013243997), (53, 0.06052729), (56, 0.012133586), (57, 0.016616648), (59, 0.19282591), (60, 0.13015571), (62, 0.018556116), (67, 0.016297748), (68, 0.13140905), (78, 0.02641064), (84, 0.044901762), (86, 0.07512327), (95, 0.021016728)]
[(15, 0.035940673), (24, 0.064974874), (50, 0.09080162), (53, 0.04392728), (56, 0.020645535), (58, 0.013512891), (60, 0.0545214), (62, 0.01100606), (65, 0.07648633), (68, 0.029317494), (78, 0.042024545), (83, 0.02220673), (84, 0.057485394), (86, 0.06801625), (99, 0.30460674)]
[(8, 0.024419), (24, 0.05923313), (30, 0.10512591), (50, 0.011844965), (53, 0.1297669), (54, 0.011499804), (55, 0.031125648), (56, 0.022731306), (57, 0.06567384), (62, 0.023078762), (65, 0.025289916), (68, 0.087160684), (78, 0.03931254), (84, 0.034407437), (86, 0.11684337), (90, 0.086222276), (95, 0.024985433)]
[(18, 0.10186765), (24, 0.020001784), (33, 0.053771555), (50, 0.018766854),

[(19, 0.031358115), (22, 0.173513), (24, 0.050182667), (33, 0.010562963), (50, 0.028435852), (53, 0.07193481), (56, 0.011095762), (57, 0.029756399), (58, 0.023111656), (60, 0.09181993), (65, 0.02413308), (68, 0.09096043), (76, 0.012534534), (78, 0.03337445), (84, 0.05571566), (86, 0.08984571), (95, 0.049555585), (96, 0.010206551), (99, 0.012139013)]
[(19, 0.011697523), (24, 0.035216242), (29, 0.3365316), (50, 0.020300286), (53, 0.10643084), (56, 0.011221357), (57, 0.027610557), (58, 0.014789461), (60, 0.07158065), (62, 0.02018492), (65, 0.023407169), (68, 0.07704683), (75, 0.01770331), (78, 0.028286584), (84, 0.03857159), (86, 0.072878286)]
[(5, 0.42771178), (15, 0.03315739), (53, 0.10497064), (55, 0.044777982), (57, 0.08238758), (60, 0.08772521), (72, 0.013615752), (76, 0.026285453), (86, 0.056876894)]
[(24, 0.047009546), (53, 0.08410159), (60, 0.08351865), (67, 0.5643162), (68, 0.046678692), (78, 0.050278727), (86, 0.05138771)]
[(18, 0.010348372), (24, 0.088985264), (27, 0.015354527)

[(2, 0.012489662), (24, 0.03802467), (50, 0.05683217), (53, 0.12137862), (55, 0.120782785), (56, 0.041718643), (57, 0.05211692), (60, 0.07792695), (78, 0.061795224), (84, 0.020156754), (86, 0.08278089), (91, 0.010178582), (94, 0.021942308), (95, 0.026290476), (96, 0.013624814), (97, 0.1412588)]
[(1, 0.015144993), (18, 0.011048953), (24, 0.01438801), (45, 0.018141398), (50, 0.042134438), (53, 0.111182965), (55, 0.035366654), (57, 0.04965342), (60, 0.10147771), (65, 0.034659028), (67, 0.044090062), (68, 0.047203995), (78, 0.03783693), (84, 0.048206434), (86, 0.10718277), (90, 0.17273897), (95, 0.033853423)]
[(24, 0.027937721), (53, 0.039286233), (60, 0.032485817), (78, 0.7668371), (86, 0.05662293)]
[(24, 0.022469454), (53, 0.085952416), (58, 0.01067085), (60, 0.017076924), (62, 0.5559892), (65, 0.017662352), (68, 0.070502505), (84, 0.029695543), (86, 0.11511584)]
[(8, 0.01759121), (24, 0.030958055), (32, 0.28262863), (50, 0.07581262), (53, 0.07677124), (55, 0.022128273), (57, 0.09634943)

[(8, 0.0101228375), (19, 0.041324504), (24, 0.028365714), (26, 0.102171), (33, 0.023500066), (42, 0.01063068), (51, 0.017094504), (53, 0.047064427), (57, 0.058643084), (60, 0.103826106), (65, 0.033380426), (68, 0.012392267), (75, 0.017292434), (78, 0.064510025), (84, 0.05169952), (86, 0.11316098), (95, 0.020042738), (97, 0.1568863)]
[(31, 0.4929736), (50, 0.030250045), (53, 0.086848125), (60, 0.05189131), (62, 0.015795257), (68, 0.06291014), (78, 0.027567364), (86, 0.14166427)]
[(24, 0.026205586), (50, 0.14172596), (53, 0.10036792), (54, 0.07098074), (55, 0.024788547), (57, 0.061116636), (60, 0.047939185), (65, 0.048644777), (68, 0.051426627), (78, 0.029171223), (83, 0.010534693), (84, 0.010449264), (86, 0.06834627), (95, 0.24014537)]
[(8, 0.010612756), (24, 0.09878529), (31, 0.045092948), (32, 0.27358708), (50, 0.023793936), (53, 0.0809364), (57, 0.041225582), (60, 0.07758256), (62, 0.0146153085), (76, 0.024567405), (78, 0.04741919), (86, 0.12651363), (95, 0.039076015)]
[(4, 0.0353089

[(19, 0.054597117), (24, 0.04437251), (29, 0.011794452), (30, 0.05264033), (50, 0.039096665), (51, 0.02528026), (53, 0.07918778), (56, 0.017341265), (57, 0.10623308), (60, 0.017417127), (64, 0.011330668), (67, 0.025609331), (68, 0.07168733), (76, 0.01421282), (78, 0.026886562), (79, 0.04107714), (84, 0.050684076), (86, 0.113333724), (90, 0.017308176), (96, 0.069926105)]
[(1, 0.03454268), (5, 0.026261665), (18, 0.023583027), (23, 0.027286936), (24, 0.14250016), (25, 0.014964688), (29, 0.04311951), (30, 0.045836255), (42, 0.035334915), (45, 0.036969583), (55, 0.16389872), (57, 0.07138901), (60, 0.095949486), (84, 0.03647665), (86, 0.055531234), (97, 0.022128489), (99, 0.011304537)]
[(24, 0.03592943), (29, 0.01004471), (30, 0.025980385), (41, 0.011361728), (50, 0.040484972), (53, 0.069077976), (54, 0.013281032), (55, 0.091560535), (57, 0.09352228), (60, 0.09352105), (65, 0.031698782), (67, 0.045985408), (68, 0.045951992), (76, 0.024473231), (78, 0.044550676), (84, 0.040160943), (86, 0.071